In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# URL of the website
URL = "https://www.redbus.in/online-booking/wbtc-ctc"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver
    

def load_page(driver, url):
    driver.get(url)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'route')))

def scrape_bus_routes(driver):
    try:
        route_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, 'route'))
        )
        bus_routes_link = [route.get_attribute('href') for route in route_elements]
        bus_routes_name = [route.text.strip() for route in route_elements]
        return bus_routes_link, bus_routes_name
    except Exception as e:
        print(f"Error occurred while scraping bus routes: {str(e)}")
        return [], []

def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "button"))
        )

        # Click the "View Buses" button
        view_buses_button = driver.find_element(By.CLASS_NAME, "button")
        driver.execute_script("arguments[0].click();", view_buses_button)
        
        # Wait until buses are visible
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "travels.lh-24.f-bold.d-color"))
        )

        # Scroll down to load more content
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Retrieve bus details
        bus_details = []
        try:
            bus_name_elements = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CLASS_NAME, "travels.lh-24.f-bold.d-color"))
            )
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text if i < len(bus_type_elements) else 'N/A',
                    "Departing_Time": departing_time_elements[i].text if i < len(departing_time_elements) else 'N/A',
                    "Duration": duration_elements[i].text if i < len(duration_elements) else 'N/A',
                    "Reaching_Time": reaching_time_elements[i].text if i < len(reaching_time_elements) else 'N/A',
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text if i < len(price_elements) else '0',
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
        except Exception as e:
            print(f"Error occurred while scraping bus details: {str(e)}")

        return bus_details

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

def scrape_all_pages():
    all_bus_details = []
    driver = initialize_driver()
    try:
        load_page(driver, URL)

        for page in range(1, 4):  # Number of pages
            try:
                if page > 1:
                    pagination_tab = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                    )
                    driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                    driver.execute_script("arguments[0].click();", pagination_tab)
                    WebDriverWait(driver, 10).until(EC.staleness_of(pagination_tab))

                all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
                for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                    bus_details = scrape_bus_details(driver, link, name)
                    if bus_details:
                        all_bus_details.extend(bus_details)

            except Exception as e:
                print(f"Error occurred while accessing page {page}: {str(e)}")

    finally:
        driver.quit()

    return all_bus_details

# Scrape routes and details from all pages
all_bus_details = scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('wb_bus_details.csv', index=False)


Error occurred while accessing https://www.redbus.in/bus-tickets/barasat-west-bengal-to-digha: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7C00F9412+29090]
	(No symbol) [0x00007FF7C006E239]
	(No symbol) [0x00007FF7BFF2B1DA]
	(No symbol) [0x00007FF7BFF7EFE7]
	(No symbol) [0x00007FF7BFF7F23C]
	(No symbol) [0x00007FF7BFFC97C7]
	(No symbol) [0x00007FF7BFFA672F]
	(No symbol) [0x00007FF7BFFC65A2]
	(No symbol) [0x00007FF7BFFA6493]
	(No symbol) [0x00007FF7BFF709D1]
	(No symbol) [0x00007FF7BFF71B31]
	GetHandleVerifier [0x00007FF7C041871D+3302573]
	GetHandleVerifier [0x00007FF7C0464243+3612627]
	GetHandleVerifier [0x00007FF7C045A417+3572135]
	GetHandleVerifier [0x00007FF7C01B5EB6+801862]
	(No symbol) [0x00007FF7C007945F]
	(No symbol) [0x00007FF7C0074FB4]
	(No symbol) [0x00007FF7C0075140]
	(No symbol) [0x00007FF7C006461F]
	BaseThreadInitThunk [0x00007FF9DFCB257D+29]
	RtlUserThreadStart [0x00007FF9E110AF28+40]

Error occurred while accessing https://www.redbus.in/bus-tickets/barasat-west-be

In [2]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,The Joy Ride,A/C Seater / Sleeper (2+2),21:15,06h 20m,03:35,3.8,520,51 Seats available
1,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,Arunita Paribahan,A/C Seater (2+3),23:35,05h 25m,05:00,3.0,INR 370,21 Seats available
2,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,Aradhana Travels,A/C Seater / Sleeper (2+2),18:35,05h 20m,23:55,1.2,475,19 Seats available
3,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,Satya Paribahan,A/C Seater (2+3),23:00,06h 10m,05:10,2.8,INR 320,31 Seats available
4,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,Snemita Paribahan (Shinjini),AC Seater (2+3),23:35,05h 25m,05:00,2.4,INR 330,29 Seats available
5,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-BANKURA - KARUNAMOYEE - 14:15 (BNK DEPOT...,Non AC Seater (2+3),16:00,04h 00m,20:00,3.5,INR 155,38 Seats available
6,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-ASANSOL - KOLKATA - VOLVO - 15:05 (BEL D...,Volvo AC Seater (2+2),16:15,04h 10m,20:25,3.6,INR 360,53 Seats available
7,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-ASANSOL - KOLKATA - 15:05 (HWH DEPOT) - 210,Non AC Seater (2+3),16:15,03h 30m,19:45,3.1,INR 150,52 Seats available
8,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-ASANSOL - KARUNAMOYEE - 15:10 (BEL DEPOT...,Non AC Seater (2+3),16:20,04h 00m,20:20,3.7,INR 155,48 Seats available
9,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-ASANSOL - KOLKATA - 15:35 (HOWRAH DEPOT)...,Non AC Seater (2+3),16:45,03h 30m,20:15,2.2,INR 150,51 Seats available


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Route_Name         46 non-null     object
 1   Route_Link         46 non-null     object
 2   Bus_Name           46 non-null     object
 3   Bus_Type           46 non-null     object
 4   Departing_Time     46 non-null     object
 5   Duration           46 non-null     object
 6   Reaching_Time      46 non-null     object
 7   Star_Rating        46 non-null     object
 8   Price              46 non-null     object
 9   Seat_Availability  46 non-null     object
dtypes: object(10)
memory usage: 3.7+ KB


In [4]:
df.head()  

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,The Joy Ride,A/C Seater / Sleeper (2+2),21:15,06h 20m,03:35,3.8,520,51 Seats available
1,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,Arunita Paribahan,A/C Seater (2+3),23:35,05h 25m,05:00,3.0,INR 370,21 Seats available
2,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,Aradhana Travels,A/C Seater / Sleeper (2+2),18:35,05h 20m,23:55,1.2,475,19 Seats available
3,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,Satya Paribahan,A/C Seater (2+3),23:00,06h 10m,05:10,2.8,INR 320,31 Seats available
4,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,Snemita Paribahan (Shinjini),AC Seater (2+3),23:35,05h 25m,05:00,2.4,INR 330,29 Seats available


In [5]:
df.isnull()

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False


In [6]:
print(df.isna().sum())

Route_Name           0
Route_Link           0
Bus_Name             0
Bus_Type             0
Departing_Time       0
Duration             0
Reaching_Time        0
Star_Rating          0
Price                0
Seat_Availability    0
dtype: int64


In [7]:
data = df.dropna()

In [8]:
import mysql.connector
conn = mysql.connector.connect(user='chakradhar', password='chakradhar123456789', host='localhost', database='bus')
cursor = conn.cursor()

In [10]:
for index, row in data.iterrows():
    cursor.execute("""
        INSERT INTO wb_bus (Route_Name, Route_Link, Bus_Name, Bus_Type, Departing_Time, Duration, Reaching_Time, Star_Rating, Price, Seats_Available)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)""",
        (row['Route_Name'], row['Route_Link'], row['Bus_Name'], row['Bus_Type'], row['Departing_Time'], row['Duration'], row['Reaching_Time'], row['Star_Rating'], row['Price'], row['Seat_Availability']))

In [11]:
conn.commit()
cursor.close()
conn.close()